In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score,recall_score,accuracy_score,precision_score,confusion_matrix
from sklearn.metrics import confusion_matrix,classification_report

import xgboost as xgb
from sklearn.metrics import accuracy_score


In [ ]:
df = pd.read_csv("/content/Final_set.csv")
df.head()

,C1,Text,Label,label_id
0,0,"Previously , the joint venture had appeared in...",Debt,0
1,1,It has an untapped debt facility of more than ...,Debt,0
2,2,The SFR deal will allow Vodafone to pay down d...,Debt,0
3,3,"Restating its books for September last year , ...",Debt,0
4,4,"Group net debt was lower than expected , thank...",Debt,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10354 entries, 0 to 10353
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   C1        10354 non-null  int64 
 1   Text      10354 non-null  object
 2   Label     10354 non-null  object
 3   label_id  10354 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 323.7+ KB


In [ ]:
df = df.drop(columns = 'C1',axis =1)
df

,Text,Label,label_id
0,"Previously , the joint venture had appeared in...",Debt,0
1,It has an untapped debt facility of more than ...,Debt,0
2,The SFR deal will allow Vodafone to pay down d...,Debt,0
3,"Restating its books for September last year , ...",Debt,0
4,"Group net debt was lower than expected , thank...",Debt,0
...,...,...,...
10349,While Wal-Mart was never in danger of going ou...,unknown,13
10350,The program requires a valid electronic benefi...,unknown,13
10351,This wo n't be easy,unknown,13
10352,"The presence of Lore , who has shown he knows ...",unknown,13


In [ ]:
x = df.drop(columns = 'label_id' ,axis = 1)
y = df["label_id"]

In [ ]:
ps = PorterStemmer()


In [ ]:
def stemming(Text): #function called as stemming
    stemmed_content = re.sub("[^a-zA-Z]"," ",Text)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content


In [ ]:
nltk.download('stopwords')
df["Text"] = df["Text"].apply(stemming)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(df["Text"])

0        previous joint ventur appear account per cent ...
1        untap debt facil pound bn use buy market recov...
2        sfr deal allow vodafon pay debt accord merril ...
3        restat book septemb last year mobil phone oper...
4        group net debt lower expect thank properti dis...
                               ...                        
10349    wal mart never danger go busi n address digit ...
10350    program requir valid electron benefit transfer...
10351                                            wo n easi
10352    presenc lore shown know take amazon give wal m...
10353    amazon shown willing make sort sacrific wal ma...
Name: Text, Length: 10354, dtype: object


In [ ]:
wordnetlemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatizer(Text): #function called as stemming
    stemmed_content = re.sub("[^a-zA-Z]"," ",Text)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [wordnetlemmatizer.lemmatize(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content

In [ ]:
 nltk.download('wordnet')
df["Text"] = df["Text"].apply(lemmatizer)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
x = df["Text"].values
y = df["label_id"].values

In [ ]:
vectorizer  =TfidfVectorizer()
vectorizer.fit(x)
x = vectorizer.transform(x)


In [ ]:
print(x)

  (0, 8994)	0.14473777242868666
  (0, 8906)	0.19764543577531074
  (0, 8526)	0.2440016047972009
  (0, 7920)	0.2600336649953867
  (0, 6736)	0.3088681252148601
  (0, 6562)	0.2276914653455888
  (0, 6412)	0.24591054118410252
  (0, 6110)	0.10634558465291906
  (0, 6018)	0.22060724190004857
  (0, 5562)	0.15206980090466582
  (0, 5544)	0.19845745563085873
  (0, 5109)	0.2662507665600595
  (0, 4376)	0.2095305245362347
  (0, 3795)	0.18282431513011094
  (0, 2584)	0.2308207110663759
  (0, 2097)	0.18391898523116162
  (0, 1350)	0.10799362559069187
  (0, 774)	0.31460501237138666
  (0, 508)	0.19985148003387834
  (0, 411)	0.23017345822581012
  (0, 53)	0.20916386048396102
  (1, 8848)	0.2018479639444494
  (1, 8796)	0.36471486739436426
  (1, 8762)	0.3416759463982313
  (1, 6746)	0.28610057548688306
  :	:
  (10352, 6996)	0.3230094175274162
  (10352, 6394)	0.3070492188341035
  (10352, 5037)	0.27901910787764594
  (10352, 4857)	0.3230094175274162
  (10352, 4533)	0.26552268878801105
  (10352, 3491)	0.2106544271647

In [ ]:
def split_data(x, y, test_size=0.2, random_state=42):
    # Split the data into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)

    return x_train, x_test, y_train, y_test

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2 ,random_state = 42)

In [ ]:
def random_forest_classifier_cv(x_train, y_train, cv=5, n_estimators=100, max_depth=2):
    global clf,scores
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    scores = cross_val_score(clf, x, y, cv=cv)
    clf.fit(x_train, y_train)

    return clf, scores
random_forest_classifier_cv(x_train, y_train, cv=5, n_estimators=100, max_depth=20)



(RandomForestClassifier(max_depth=20),
 array([0.62530179, 0.6088846 , 0.59439884, 0.51279575, 0.51449275]))

In [ ]:
x_train_pred = clf.predict(x_train)
training_data_acc = accuracy_score(x_train_pred,y_train)

In [ ]:
print("training acuuracy is :",training_data_acc)

training acuuracy is : 0.6111312326451769


In [ ]:
x_test_pred = clf.predict(x_test)
testing_data_acc  = accuracy_score(x_test_pred,y_test)

In [ ]:
print("testing acuuracy is :",testing_data_acc)


testing acuuracy is : 0.570738773539353


In [ ]:
# Testing
y_pred = clf.predict(x_test)
cnf_matrix = confusion_matrix(y_test,y_pred)
print("confusion matrix\n",cnf_matrix)
print("*"*20)
accuracy = accuracy_score(y_test,y_pred)
print("accuracy",accuracy)
print("*"*20)
clf_report = classification_report(y_test,y_pred)
print("classificatio report\n",clf_report)

confusion matrix
 [[   0    0    0    0    0    0    0    0    0    0    0    0    0   16]
 [   0    1    0    3    0    0    0    0    0    0    0    0    0   43]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   31]
 [   0    0    0  162    0    0    0    0    0    0    0    0    0  266]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   32]
 [   0    0    0    0    0    2    0    0    0    0    0    0    0   32]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   81]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   84]
 [   0    0    0    0    0    0    0    0    4    0    0    0    0   85]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   92]
 [   0    0    0    0    0    0    0    0    0    0    4    0    0   79]
 [   0    0    0    8    0    0    0    0    0    0    0    0    0   19]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    9]
 [   0    0    0    8    0    0  

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Training
y_pred_train = clf.predict(x_train)
cnf_matrix = confusion_matrix(y_train,y_pred_train)
print("confusion matrix\n",cnf_matrix)
print("*"*20)
accuracy = accuracy_score(y_train,y_pred_train)
print("accuracy",accuracy)
print("*"*20)
clf_report = classification_report(y_train,y_pred_train)
print("classificatio report\n",clf_report)

confusion matrix
 [[   0    0    0    0    0    0    0    0    0    0    0    0    0   39]
 [   0   24    0    3    0    0    0    0    0    0    0    0    0  177]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0  108]
 [   0    0    0  769    0    0    0    0    0    0    0    0    0  923]
 [   0    0    0    0    2    0    0    0    0    0    0    0    0  213]
 [   0    0    0    0    0    9    0    0    0    0    0    0    0   95]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0  258]
 [   0    0    0    1    0    0    0    6    0    0    0    0    0  313]
 [   0    0    0    0    0    0    0    0   27    0    0    0    0  294]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0  388]
 [   0    0    0    0    0    0    0    0    0    0   37    0    0  257]
 [   0    0    0   17    0    0    0    0    0    0    0   11    0  100]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   32]
 [   0    0    0    3    0    0  

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

def xgb_classifier(x_train, y_train, x_test, y_test):
    # Define the XGBoost classifier
    xgb_model = xgb.XGBClassifier()

    # Define the hyperparameters grid
    param_grid = {'n_estimators': [50, 100, 200],
                  'max_depth': [3, 5, 7],
                  'learning_rate': [0.1, 0.01]}

    # Perform a grid search with cross-validation
    grid_search = GridSearchCV(xgb_model, param_grid, cv=5)
    grid_search.fit(x_train, y_train)

    # Get the best model and make predictions on the testing data
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test)

    # Calculate accuracy score
    accuracy = accuracy_score(y_test, y_pred)

    return best_model, accuracy

In [ ]:
best_model, accuracy = xgb_classifier(x_train, y_train, x_test, y_test)

In [ ]:
x_test_pred = clf.predict(x_test)
testing_data_acc  = accuracy_score(x_test_pred,y_test)

In [ ]:
# Train the SVM model using cross-validation
from sklearn import svm
def svm_text_classification(x_train,y_train,c=1.0,kernel='linear',gamma='scale'):
    global clf,scores
    clf = Svm.SVC(C=c,kernel=kernel,gamma=gamma)
    scores = cross_val_score(clf, x, y, cv=5)
    clf.fit(x_train,y_train)
    return clf,scores
    svm_text_classification(x_train,y_train,c=1.0,kernel='linear',gamma='scale')

    y_pred=clf.predict(x_test)# Predict the class labels for the test data


In [ ]:
x_train_pred = clf.predict(x_train)
training_data_acc = accuracy_score(x_train_pred,y_train)

In [ ]:
print("training acuuracy is :",training_data_acc)

training acuuracy is : 0.6111312326451769


In [ ]:
print("testing acuuracy is :",testing_data_acc)


testing acuuracy is : 0.570738773539353


In [ ]:
# Testing
y_pred = clf.predict(x_test)
cnf_matrix = confusion_matrix(y_test,y_pred)
print("confusion matrix\n",cnf_matrix)
print("*"*20)
accuracy = accuracy_score(y_test,y_pred)
print("accuracy",accuracy)
print("*"*20)
clf_report = classification_report(y_test,y_pred)
print("classificatio report\n",clf_report)

confusion matrix
 [[   0    0    0    0    0    0    0    0    0    0    0    0    0   16]
 [   0    1    0    3    0    0    0    0    0    0    0    0    0   43]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   31]
 [   0    0    0  162    0    0    0    0    0    0    0    0    0  266]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   32]
 [   0    0    0    0    0    2    0    0    0    0    0    0    0   32]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   81]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   84]
 [   0    0    0    0    0    0    0    0    4    0    0    0    0   85]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   92]
 [   0    0    0    0    0    0    0    0    0    0    4    0    0   79]
 [   0    0    0    8    0    0    0    0    0    0    0    0    0   19]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    9]
 [   0    0    0    8    0    0  

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Train the Naive Bayes model using cross-validation
from sklearn.naive_bayes import MultinomialNB
# Define the Naive Bayes classifier object
def text_classification_nb_cv(x_train, y_train, n_folds=5,n_estimators=100, max_depth=2):
   global clf,scores
   clf = MultinomialNB(n_estimators=n_estimators, max_depth=max_depth)
   scores= cross_val_score(clf, x, y, cv=n_folds)
   return clf, scores
   text_classification_nb_cv(x_train, y_train, cv=5, n_estimators=100, max_depth=20)
   clf.fit(x_train, y_train)# Train the classifier on the training data
   y_pred = clf.predict(x_test)# Predict the class labels for the test data

In [ ]:
x_train_pred = clf.predict(x_train)
training_data_acc = accuracy_score(x_train_pred,y_train)

In [ ]:
print("training acuuracy is :",training_data_acc)

training acuuracy is : 0.6111312326451769


In [ ]:
print("testing acuuracy is :",testing_data_acc)

testing acuuracy is : 0.570738773539353


In [ ]:
# Testing
y_pred = clf.predict(x_test)
cnf_matrix = confusion_matrix(y_test,y_pred)
print("confusion matrix\n",cnf_matrix)
print("*"*20)
accuracy = accuracy_score(y_test,y_pred)
print("accuracy",accuracy)
print("*"*20)
clf_report = classification_report(y_test,y_pred)
print("classificatio report\n",clf_report)

confusion matrix
 [[   0    0    0    0    0    0    0    0    0    0    0    0    0   16]
 [   0    1    0    3    0    0    0    0    0    0    0    0    0   43]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   31]
 [   0    0    0  162    0    0    0    0    0    0    0    0    0  266]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   32]
 [   0    0    0    0    0    2    0    0    0    0    0    0    0   32]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   81]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   84]
 [   0    0    0    0    0    0    0    0    4    0    0    0    0   85]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   92]
 [   0    0    0    0    0    0    0    0    0    0    4    0    0   79]
 [   0    0    0    8    0    0    0    0    0    0    0    0    0   19]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    9]
 [   0    0    0    8    0    0  

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
